In [ ]:
!pip install biopython

from Bio import Entrez

# Configure ton adresse e-mail
Entrez.email = "worthylastchance@gmail.com"

def search_pubmed(query, max_results=10):
    results = []
    try:
        # Recherche dans PubMed
        handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
        record = Entrez.read(handle)

        # Récupérer les détails des articles trouvés
        for article_id in record['IdList']:
            details = fetch_article_details([article_id])  # Récupérer les détails de l'article
            if details:
                results.append(details[0])  # Ajoute les détails du premier article
    except Exception as e:
        print(f"Erreur lors de la recherche sur PubMed: {e}")

    return results

def fetch_article_details(article_ids):
    articles = []
    for article_id in article_ids:
        try:
            # Récupérer les détails de l'article
            handle = Entrez.efetch(db="pubmed", id=article_id, retmode="xml")
            record = Entrez.read(handle)
            for article in record['PubmedArticle']:
                title = article['MedlineCitation']['Article']['ArticleTitle']
                abstract = article['MedlineCitation']['Article']['Abstract']['AbstractText'][0]

                # Ajoute les détails de l'article
                articles.append({
                    'id': article_id,
                    'title': title,
                    'abstract': abstract,
                    'source': 'PubMed',
                    'similar_articles': fetch_similar_articles(article_id)  # Récupérer les articles similaires
                })
        except Exception as e:
            print(f"Erreur lors de la récupération de l'article {article_id}: {e}")
    return articles

def fetch_similar_articles(article_id):
    # Cette fonction devrait récupérer des articles similaires à l'article donné
    # Ici, nous ajoutons des exemples pour illustrer le format
    similar_articles = []
    for i in range(5):  # Limiter à 5 articles similaires
        similar_articles.append({
            'id': f"similar_{article_id}_{i}",
            'title': f"Titre d'article similaire {i+1}",
            'abstract': f"Résumé de l'article similaire {i+1}."
        })
    return similar_articles

def main(query):
    pubmed_results = search_pubmed(query)

    for result in pubmed_results:
        print(f"ID: {result['id']}, Title: {result['title']}\n")
        print(f"Abstract: {result['abstract']}\n")  # Ajout d'un saut de ligne ici
        print(f"Source: {result['source']}")
        print("Similar Articles:")
        for similar in result['similar_articles']:
            print(f"  - Similar ID: {similar['id']}, Title: {similar['title']}, Abstract: {similar['abstract']}")
        print()  # Ligne vide pour séparer les articles

# Exécution de la recherche
query = "liposomal soft mist inhalers"  # Exemple de requête
main(query)



ID: 38237688, Title: Tunable rigidity of PLGA shell-lipid core nanoparticles for enhanced pulmonary siRNA delivery in 2D and 3D lung cancer cell models.

Abstract: Faced with the threat of lung cancer-related deaths worldwide, small interfering RNA (siRNA) can silence tumor related messenger RNA (mRNA) to tackle the issue of drug resistance with enhanced anti-tumor effects. However, how to increase lung tumor targeting and penetration with enhanced gene silencing are the issues to be addressed. Thus, the objective of this study is to explore the feasibility of designing non-viral siRNA vectors for enhanced lung tumor therapy via inhalation. Here, shell-core based polymer-lipid hybrid nanoparticles (HNPs) were prepared via microfluidics by coating PLGA on siRNA-loaded cationic liposomes (Lipoplexes). Transmission electron microscopy and energy dispersive spectroscopy study demonstrated that HNP consists of a PLGA shell and a lipid core. Atomic force microscopy study indicated that the r

In [ ]:
!pip install biopython requests
!pip install scholarly

from Bio import Entrez
import requests
from scholarly import scholarly



# Configure ton adresse e-mail
Entrez.email = "worthylastchance@gmail.com"

def search_pubmed(query, max_results=10):
    results = []
    try:
        handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
        record = Entrez.read(handle)

        for article_id in record['IdList']:
            details = fetch_pubmed_details(article_id)  # Récupérer les détails de l'article
            if details:
                results.append(details)  # Ajoute les détails de l'article
    except Exception as e:
        print(f"Erreur lors de la recherche sur PubMed: {e}")

    return results

def fetch_pubmed_details(article_id):
    try:
        handle = Entrez.efetch(db="pubmed", id=article_id, retmode="xml")
        record = Entrez.read(handle)
        article = record['PubmedArticle'][0]  # Assure-toi de récupérer le premier article
        title = article['MedlineCitation']['Article']['ArticleTitle']
        abstract = article['MedlineCitation']['Article'].get('Abstract', {}).get('AbstractText', ['Aucune description disponible'])[0]

        return {
            'id': article_id,
            'title': title,
            'abstract': abstract,
            'source': 'PubMed',
            'similar_articles': fetch_similar_articles(article_id)  # Récupérer les articles similaires
        }
    except Exception as e:
        print(f"Erreur lors de la récupération de l'article {article_id}: {e}")
        return None

def fetch_similar_articles(article_id):
    # Simulation d'articles similaires pour l'exemple
    similar_articles = []
    for i in range(5):  # Limiter à 5 articles similaires
        similar_articles.append({
            'id': f"similar_{article_id}_{i}",
            'title': f"Titre d'article similaire {i+1}",
            'abstract': f"Résumé de l'article similaire {i+1}."
        })
    return similar_articles


import requests

def search_crossref(query, max_results=5):
    results = []
    try:
        url = f"https://api.crossref.org/works?query={query}&rows={max_results}"
        response = requests.get(url)
        data = response.json()

        for item in data['message']['items']:
            results.append({
                'id': item['DOI'],
                'title': item['title'][0],
                'abstract': item.get('abstract', 'Aucune description disponible'),
                'source': 'CrossRef',
                'similar_articles': []  # Simuler les articles similaires
            })
    except Exception as e:
        print(f"Erreur lors de la recherche sur CrossRef: {e}")

    return results

def remove_duplicates(results):
    seen_titles = set()  # To store titles that have already been seen
    unique_results = []

    for result in results:
        title = result['title'].lower().strip()  # Normalize title (lowercase and remove extra spaces)
        if title not in seen_titles:
            unique_results.append(result)
            seen_titles.add(title)  # Mark the title as seen

    return unique_results


def main(query):
    pubmed_results = search_pubmed(query)
    crossref_results = search_crossref(query)

    all_results = pubmed_results + crossref_results

    # Remove duplicate articles
    unique_results = remove_duplicates(all_results)

    for result in all_results:
        print(f"ID: {result['id']}, Title: {result['title']}\n")
        print(f"Abstract: {result['abstract']}\n")  # Ajout d'un saut de ligne ici
        print(f"Source: {result['source']}")
        print("Similar Articles:")
        for similar in result['similar_articles']:
            print(f"  - Similar ID: {similar['id']}, Title: {similar['title']}, Abstract: {similar['abstract']}")
        print()  # Ligne vide pour séparer les articles

# Exécution de la recherche
query = "liposomal soft mist inhalers"  # Exemple de requête
main(query)


ID: 38237688, Title: Tunable rigidity of PLGA shell-lipid core nanoparticles for enhanced pulmonary siRNA delivery in 2D and 3D lung cancer cell models.

Abstract: Faced with the threat of lung cancer-related deaths worldwide, small interfering RNA (siRNA) can silence tumor related messenger RNA (mRNA) to tackle the issue of drug resistance with enhanced anti-tumor effects. However, how to increase lung tumor targeting and penetration with enhanced gene silencing are the issues to be addressed. Thus, the objective of this study is to explore the feasibility of designing non-viral siRNA vectors for enhanced lung tumor therapy via inhalation. Here, shell-core based polymer-lipid hybrid nanoparticles (HNPs) were prepared via microfluidics by coating PLGA on siRNA-loaded cationic liposomes (Lipoplexes). Transmission electron microscopy and energy dispersive spectroscopy study demonstrated that HNP consists of a PLGA shell and a lipid core. Atomic force microscopy study indicated that the r

In [ ]:
!pip install biopython requests
!pip install scholarly

from Bio import Entrez
import requests
from scholarly import scholarly



# Configure ton adresse e-mail
Entrez.email = "blinkworth25@gmail.com"

def search_pubmed(query, max_results=10):
    results = []
    try:
        handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
        record = Entrez.read(handle)

        for article_id in record['IdList']:
            details = fetch_pubmed_details(article_id)  # Récupérer les détails de l'article
            if details:
                results.append(details)  # Ajoute les détails de l'article
    except Exception as e:
        print(f"Erreur lors de la recherche sur PubMed: {e}")

    return results

def fetch_pubmed_details(article_id):
    try:
        handle = Entrez.efetch(db="pubmed", id=article_id, retmode="xml")
        record = Entrez.read(handle)
        article = record['PubmedArticle'][0]  # Assure-toi de récupérer le premier article
        title = article['MedlineCitation']['Article']['ArticleTitle']
        abstract = article['MedlineCitation']['Article'].get('Abstract', {}).get('AbstractText', ['Aucune description disponible'])[0]

        return {
            'id': article_id,
            'title': title,
            'abstract': abstract,
            'source': 'PubMed',
            'similar_articles': fetch_similar_articles(article_id)  # Récupérer les articles similaires
        }
    except Exception as e:
        print(f"Erreur lors de la récupération de l'article {article_id}: {e}")
        return None

def fetch_similar_articles(article_id):
    # Simulation d'articles similaires pour l'exemple
    similar_articles = []
    for i in range(5):  # Limiter à 5 articles similaires
        similar_articles.append({
            'id': f"similar_{article_id}_{i}",
            'title': f"Titre d'article similaire {i+1}",
            'abstract': f"Résumé de l'article similaire {i+1}."
        })
    return similar_articles

def search_crossref(query, max_results=5):
    results = []
    try:
        url = f"https://api.crossref.org/works?query={query}&rows={max_results}"
        response = requests.get(url)
        data = response.json()

        for item in data['message']['items']:
            results.append({
                'id': item['DOI'],
                'title': item['title'][0],
                'abstract': item.get('abstract', 'Aucune description disponible'),
                'source': 'CrossRef',
                'similar_articles': []  # Simuler les articles similaires
            })
    except Exception as e:
        print(f"Erreur lors de la recherche sur CrossRef: {e}")

    return results

import time
from scholarly import scholarly
import requests

def search_google_scholar(query, max_results=5):
    results = []
    try:
        search_query = scholarly.search_pubs(query)
        for i in range(max_results):
            article = next(search_query)
            article_bib = article.get('bib', {})
            results.append({
                'id': article_bib.get('pub_year', 'Unknown'),  # Scholar doesn't use IDs
                'title': article_bib.get('title', 'No title available'),
                'abstract': article_bib.get('abstract', 'No abstract available'),
                'source': 'Google Scholar',
                'similar_articles': []  # Not available in Scholar results
            })
            time.sleep(5)  # Add a 5-second delay between each query to avoid getting blocked
    except Exception as e:
        print(f"Error searching Google Scholar: {e}")

    return results

def main(query):
    pubmed_results = search_pubmed(query)
    google_scholar_results = search_google_scholar(query)
    all_results = pubmed_results
    for result in all_results:
        print(f"ID: {result['id']}, Title: {result['title']}\n")
        print(f"Abstract: {result['abstract']}\n")
        print(f"Source: {result['source']}")
        print("Similar Articles:")
        for similar in result['similar_articles']:
            print(f"  - Similar Title: {similar['title']}")
        print()  # Ligne vide pour séparer les articles


def main(query):
    pubmed_results = search_pubmed(query)
    crossref_results = search_crossref(query)
    google_scholar_results = search_google_scholar(query)

    all_results = pubmed_results + crossref_results + google_scholar_results


    for result in all_results:
        print(f"ID: {result['id']}, Title: {result['title']}\n")
        print(f"Abstract: {result['abstract']}\n")  # Ajout d'un saut de ligne ici
        print(f"Source: {result['source']}")
        print("Similar Articles:")
        for similar in result['similar_articles']:
            print(f"  - Similar ID: {similar['id']}, Title: {similar['title']}, Abstract: {similar['abstract']}")
        print()  # Ligne vide pour séparer les articles

# Exécution de la recherche
query = "liposomal soft mist inhalers"  # Exemple de requête
main(query)

Error searching Google Scholar: 'dict' object has no attribute 'proxy_mode'
ID: 38237688, Title: Tunable rigidity of PLGA shell-lipid core nanoparticles for enhanced pulmonary siRNA delivery in 2D and 3D lung cancer cell models.

Abstract: Faced with the threat of lung cancer-related deaths worldwide, small interfering RNA (siRNA) can silence tumor related messenger RNA (mRNA) to tackle the issue of drug resistance with enhanced anti-tumor effects. However, how to increase lung tumor targeting and penetration with enhanced gene silencing are the issues to be addressed. Thus, the objective of this study is to explore the feasibility of designing non-viral siRNA vectors for enhanced lung tumor therapy via inhalation. Here, shell-core based polymer-lipid hybrid nanoparticles (HNPs) were prepared via microfluidics by coating PLGA on siRNA-loaded cationic liposomes (Lipoplexes). Transmission electron microscopy and energy dispersive spectroscopy study demonstrated that HNP consists of a PLG

In [ ]:
!pip install biopython requests
!pip install scholarly

from Bio import Entrez
import requests
from scholarly import scholarly

# Configure ton adresse e-mail
Entrez.email = "worthylastchance@gmail.com"

def search_pubmed(query, max_results=10):
    results = []
    try:
        handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
        record = Entrez.read(handle)

        for article_id in record['IdList']:
            details = fetch_pubmed_details(article_id)  # Récupérer les détails de l'article
            if details:
                results.append(details)  # Ajoute les détails de l'article
    except Exception as e:
        print(f"Erreur lors de la recherche sur PubMed: {e}")

    return results

def fetch_pubmed_details(article_id):
    try:
        handle = Entrez.efetch(db="pubmed", id=article_id, retmode="xml")
        record = Entrez.read(handle)
        article = record['PubmedArticle'][0]  # Assure-toi de récupérer le premier article
        title = article['MedlineCitation']['Article']['ArticleTitle']
        abstract = article['MedlineCitation']['Article'].get('Abstract', {}).get('AbstractText', ['Aucune description disponible'])[0]

        return {
            'id': article_id,
            'title': title,
            'abstract': abstract,
            'source': 'PubMed',
            'similar_articles': fetch_similar_articles(article_id)  # Récupérer les articles similaires
        }
    except Exception as e:
        print(f"Erreur lors de la récupération de l'article {article_id}: {e}")
        return None

def fetch_similar_articles(article_id):
    # Simulation d'articles similaires pour l'exemple
    similar_articles = []
    for i in range(5):  # Limiter à 5 articles similaires
        similar_articles.append({
            'id': f"similar_{article_id}_{i}",
            'title': f"Titre d'article similaire {i+1}",
            'abstract': f"Résumé de l'article similaire {i+1}."
        })
    return similar_articles

def search_crossref(query, max_results=5):
    results = []
    try:
        url = f"https://api.crossref.org/works?query={query}&rows={max_results}"
        response = requests.get(url)
        data = response.json()

        for item in data['message']['items']:
            results.append({
                'id': item['DOI'],
                'title': item['title'][0],
                'abstract': item.get('abstract', 'Aucune description disponible'),
                'source': 'CrossRef',
                'similar_articles': []  # Simuler les articles similaires
            })
    except Exception as e:
        print(f"Erreur lors de la recherche sur CrossRef: {e}")

    return results

def search_google_scholar(query, max_results=5):
    results = []
    try:
        search_query = scholarly.search_pubs(query)
        for i in range(max_results):
            article = next(search_query)
            article_bib = article.get('bib', {})
            results.append({
                'id': article_bib.get('pub_year', 'Unknown'),  # Scholar doesn't use IDs
                'title': article_bib.get('title', 'No title available'),
                'abstract': article_bib.get('abstract', 'No abstract available'),
                'source': 'Google Scholar',
                'similar_articles': []  # Not available in Scholar results
            })
    except Exception as e:
        print(f"Error searching Google Scholar: {e}")

    return results

def main(query):
    pubmed_results = search_pubmed(query)
    crossref_results = search_crossref(query)
    google_scholar_results = search_google_scholar(query)

    all_results = pubmed_results + crossref_results + google_scholar_results


    for result in all_results:
        print(f"ID: {result['id']}, Title: {result['title']}\n")
        print(f"Abstract: {result['abstract']}\n")  # Ajout d'un saut de ligne ici
        print(f"Source: {result['source']}")
        print("Similar Articles:")
        for similar in result['similar_articles']:
            print(f"  - Similar ID: {similar['id']}, Title: {similar['title']}, Abstract: {similar['abstract']}")
        print()  # Ligne vide pour séparer les articles

# Exécution de la recherche
query = "liposomal soft mist inhalers"  # Exemple de requête
main(query)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 

In [ ]:
!pip install biopython requests
!pip install scholarly
!pip install drugbank-downloader
!pip install ieee-xplore-api

from Bio import Entrez
import requests
from scholarly import scholarly



# Configure ton adresse e-mail
Entrez.email = "blinkworth25@gmail.com"

def search_pubmed(query, max_results=10):
    results = []
    try:
        handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
        record = Entrez.read(handle)

        for article_id in record['IdList']:
            details = fetch_pubmed_details(article_id)  # Récupérer les détails de l'article
            if details:
                results.append(details)  # Ajoute les détails de l'article
    except Exception as e:
        print(f"Erreur lors de la recherche sur PubMed: {e}")

    return results

def fetch_pubmed_details(article_id):
    try:
        handle = Entrez.efetch(db="pubmed", id=article_id, retmode="xml")
        record = Entrez.read(handle)
        article = record['PubmedArticle'][0]  # Assure-toi de récupérer le premier article
        title = article['MedlineCitation']['Article']['ArticleTitle']
        abstract = article['MedlineCitation']['Article'].get('Abstract', {}).get('AbstractText', ['Aucune description disponible'])[0]

        return {
            'id': article_id,
            'title': title,
            'abstract': abstract,
            'source': 'PubMed',
            'similar_articles': fetch_similar_articles(article_id)  # Récupérer les articles similaires
        }
    except Exception as e:
        print(f"Erreur lors de la récupération de l'article {article_id}: {e}")
        return None

def fetch_similar_articles(article_id):
    # Simulation d'articles similaires pour l'exemple
    similar_articles = []
    for i in range(5):  # Limiter à 5 articles similaires
        similar_articles.append({
            'id': f"similar_{article_id}_{i}",
            'title': f"Titre d'article similaire {i+1}",
            'abstract': f"Résumé de l'article similaire {i+1}."
        })
    return similar_articles



def search_ieee(query, max_results=5):
    results = []
    try:
        url = f"https://ieeexploreapi.ieee.org/api/v1/search/articles?queryText={query}&maxResults={max_results}&apiKey=tmaeakyytdvz63zft2crbxn6"
        response = requests.get(url)
        data = response.json()

        for item in data['articles']:
            results.append({
                'id': item['articleNumber'],
                'title': item['title'],
                'abstract': item['abstract'],
                'source': 'IEEE',
                'similar_articles': []  # Simuler les articles similaires
            })
    except Exception as e:
        print(f"Erreur lors de la recherche sur IEEE: {e}")

    return results

import requests

def search_drugbank(query, max_results=5):
    results = []
    try:
        url = f"https://api.drugbank.com/v1/us/drugs?search={query}&limit={max_results}"
        headers = {
            'Authorization': 'Bearer YOUR_API_KEY'  # Remplace par ta clé API DrugBank
        }
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            for item in data['drugs']:
                results.append({
                    'id': item['drugbank_id'],
                    'title': item['name'],
                    'abstract': item.get('description', 'Aucune description disponible'),
                    'source': 'DrugBank',
                    'similar_articles': []  # Simuler les articles similaires
                })
        else:
            print(f"Erreur lors de la recherche sur DrugBank: {response.status_code}")

    except Exception as e:
        print(f"Erreur lors de la recherche sur DrugBank: {e}")

    return results


def search_crossref(query, max_results=5):
    results = []
    try:
        url = f"https://api.crossref.org/works?query={query}&rows={max_results}"
        response = requests.get(url)
        data = response.json()

        for item in data['message']['items']:
            results.append({
                'id': item['DOI'],
                'title': item['title'][0],
                'abstract': item.get('abstract', 'Aucune description disponible'),
                'source': 'CrossRef',
                'similar_articles': []  # Simuler les articles similaires
            })
    except Exception as e:
        print(f"Erreur lors de la recherche sur CrossRef: {e}")

    return results

from scholarly import scholarly

def search_google_scholar(query, max_results=5):
    results = []
    try:
        search_query = scholarly.search_pubs(query)
        for i in range(max_results):
            article = next(search_query)
            article_bib = article.get('bib', {})
            results.append({
                'id': article_bib.get('pub_year', 'Unknown'),  # Scholar doesn't use IDs
                'title': article_bib.get('title', 'No title available'),
                'abstract': article_bib.get('abstract', 'No abstract available'),
                'source': 'Google Scholar',
                'similar_articles': []  # Not available in Scholar results
            })
    except Exception as e:
        print(f"Error searching Google Scholar: {e}")

    return results


def main(query):
    pubmed_results = search_pubmed(query)
    google_scholar_results = search_google_scholar(query)

    all_results = pubmed_results + google_scholar_results
    for result in all_results:
        print(f"ID: {result['id']}, Title: {result['title']}\n")
        print(f"Abstract: {result['abstract']}\n")
        print(f"Source: {result['source']}")
        print("Similar Articles:")
        for similar in result['similar_articles']:
            print(f"  - Similar Title: {similar['title']}")
        print()  # Ligne vide pour séparer les articles


def main(query):
    pubmed_results = search_pubmed(query)
    crossref_results = search_crossref(query)
    ieee_results = search_ieee(query)
    drugbank_results = search_drugbank(query)
    google_scholar_results = search_google_scholar(query)

    all_results = pubmed_results + crossref_results + ieee_results + drugbank_results + google_scholar_results


    for result in all_results:
        print(f"ID: {result['id']}, Title: {result['title']}\n")
        print(f"Abstract: {result['abstract']}\n")  # Ajout d'un saut de ligne ici
        print(f"Source: {result['source']}")
        print("Similar Articles:")
        for similar in result['similar_articles']:
            print(f"  - Similar ID: {similar['id']}, Title: {similar['title']}, Abstract: {similar['abstract']}")
        print()  # Ligne vide pour séparer les articles

# Exécution de la recherche
query = "liposomal soft mist inhalers"  # Exemple de requête
main(query)

ERROR: Could not find a version that satisfies the requirement ieee-xplore-api (from versions: none)
ERROR: No matching distribution found for ieee-xplore-api
Erreur lors de la recherche sur IEEE: Expecting value: line 1 column 1 (char 0)
Erreur lors de la recherche sur DrugBank: 401
Error searching Google Scholar: 'dict' object has no attribute 'proxy_mode'
ID: 38237688, Title: Tunable rigidity of PLGA shell-lipid core nanoparticles for enhanced pulmonary siRNA delivery in 2D and 3D lung cancer cell models.

Abstract: Faced with the threat of lung cancer-related deaths worldwide, small interfering RNA (siRNA) can silence tumor related messenger RNA (mRNA) to tackle the issue of drug resistance with enhanced anti-tumor effects. However, how to increase lung tumor targeting and penetration with enhanced gene silencing are the issues to be addressed. Thus, the objective of this study is to explore the feasibility of designing non-viral siRNA vectors for enhanced lung tumor therapy via in